# TP : Simplified MapReduce

In this scenario, imagine you’re working for a popular e-commerce site.

Understanding when traffic is at its peak helps inform decisions about server scaling, marketing strategies, and customer support. The MapReduce job here processes large volumes of server logs to extract hourly traffic counts, providing insights into which times see the most user activity.

By analyzing page visit counts, the company can optimize popular pages, adjust advertising placements, and improve user experience on high-traffic pages. This MapReduce job counts the number of times each URL is accessed, revealing the most visited pages.

Understanding user engagement helps tailor site features and marketing efforts. The goal is to calculate the mean session duration for each unique user, indicating how long users are staying and engaging with the website’s content.

By tracking error response codes (like 404 or 500) associated with URLs, the company can prioritize bug fixes and improve user experience on critical pages. This analysis supports backend optimization by isolating pages where users encounter issues.


1. "Using the traffic data, at what time does the website experience its peak load, and how can this insight guide server capacity planning?"

Mapper: Extracts the hour from each timestamp and emits a (hour, 1) pair.
Reducer: Aggregates the counts by hour, outputting {hour: request_count, ...} to show peak traffic times.

2. "What are the top 3 most-visited pages on the site, and how could this information affect design decisions?"

Mapper: Maps each URL from a log entry to (url, 1).
Reducer: Aggregates counts per URL, resulting in {url: visit_count, ...} for a popularity ranking of pages.

3. "Which users show the highest average session durations, and what might this suggest about their engagement levels?"

Mapper: Maps each user ID to (user_id, session_duration) from each log entry.
Reducer: Aggregates the durations by user, providing {user_id: average_duration, ...} for understanding user engagement patterns.

4. "Which URLs have the highest error rates, and how should the engineering team prioritize fixes?"

Mapper: Maps URLs with error response codes (>=400) to (url, 1).
Reducer: Counts errors per URL, yielding {url: error_count, ...} to identify problematic pages needing attention.

Since we’re using Google Colab with limited resources, the MapReduce operations can be simulated simulated, applying the mapper and reducer logic, and printing the results.

In [1]:
import random
from datetime import datetime, timedelta
import pandas as pd

# Parameters
num_records = 10000
user_ids = [f"user_{i:03}" for i in range(1, 101)]
urls = ["/home", "/products", "/contact", "/about", "/cart", "/checkout"]
response_codes = [200, 200, 200, 404, 500]  # Higher chance of 200 for realistic data
ip_addresses = [f"192.168.1.{i}" for i in range(1, 101)]
user_agents = ["Mozilla/5.0", "Chrome/90.0", "Safari/537.36", "Edge/85.0"]

# Function to generate random timestamp
def random_timestamp(start, end):
    return (start + timedelta(seconds=random.randint(0, int((end - start).total_seconds())))).isoformat()

# Data generation
data = []
start_time = datetime(2023, 1, 1)
end_time = datetime(2023, 10, 1)

for _ in range(num_records):
    log_entry = {
        "timestamp": random_timestamp(start_time, end_time),
        "user_id": random.choice(user_ids),
        "ip_address": random.choice(ip_addresses),
        "url": random.choice(urls),
        "response_code": random.choice(response_codes),
        "session_duration": random.randint(10, 300),  # in seconds
        "user_agent": random.choice(user_agents)
    }
    data.append(log_entry)


TODO : convert "data" list to pandas DataFrame

In [2]:
#df =

TODO :

Peak Traffic Analysis: Identify the busiest hours.

Most Visited Pages: Count visits per URL.

Average Session Duration: Calculate average session duration per user.

Error Rate: Track and analyze URLs with high error response rates (e.g., 404 or 500).

Fill in the blanks in the code

In [3]:
from datetime import datetime
from collections import defaultdict

def traffic_mapper(line):
    fields = line.strip().split(",")
    timestamp = fields[0]
    #hour =
    yield (hour, 1)

def traffic_reducer(mapped_data):
    hour_counts = defaultdict(int)
    for hour, count in mapped_data:
        hour_counts[hour] += count
    return hour_counts


In [4]:
def url_mapper(line):
    fields = line.strip().split(",")
    url = fields[3]
    yield (url, 1)

def url_reducer(mapped_data):
    url_counts = defaultdict(int)
    for url, count in mapped_data:
        url_counts[url] += count
    return url_counts

In [5]:
def session_mapper(line):
    fields = line.strip().split(",")
    user_id = fields[1]
    session_duration = int(fields[5])
    yield (user_id, session_duration)

def session_reducer(mapped_data):
    user_sessions = defaultdict(list)
    for user_id, duration in mapped_data:
        user_sessions[user_id].append(duration)
    # Calculate average per user
    # user_avg_duration =
    return user_avg_duration


In [6]:
def error_mapper(line):
    fields = line.strip().split(",")
    url = fields[3]
    response_code = int(fields[4])
    if response_code >= 400:  # Considering 400 and 500 series as errors
        yield (url, 1)

def error_reducer(mapped_data):
    error_counts = defaultdict(int)
    for url, count in mapped_data:
        error_counts[url] += count
    return error_counts

In [7]:
# Run each MapReduce job

Peak Traffic Analysis
mapped_data = [(hour, 1) for hour in df['timestamp'].apply(lambda x: pd.to_datetime(x).hour)]
peak_traffic = traffic_reducer(mapped_data)
print("Peak Traffic per Hour:", peak_traffic)

# Most Visited Pages
mapped_data = [(url, 1) for url in df['url']]
most_visited = url_reducer(mapped_data)
print("Most Visited Pages:", most_visited)

# Average Session Duration per User
mapped_data = [(user_id, duration) for user_id, duration in zip(df['user_id'], df['session_duration'])]
avg_session = session_reducer(mapped_data)
print("Average Session Duration per User:", avg_session)

# Error Rate Analysis
mapped_data = [(url, 1) for url, response_code in zip(df['url'], df['response_code']) if response_code >= 400]
error_rates = error_reducer(mapped_data)
print("Error Rate per URL:", error_rates)

Peak Traffic per Hour: defaultdict(<class 'int'>, {11: 451, 0: 393, 3: 412, 15: 426, 14: 397, 23: 403, 6: 391, 4: 429, 9: 443, 8: 438, 17: 408, 22: 425, 2: 447, 20: 423, 16: 395, 18: 405, 5: 395, 13: 434, 21: 407, 19: 362, 7: 416, 1: 417, 12: 450, 10: 433})
Most Visited Pages: defaultdict(<class 'int'>, {'/cart': 1608, '/contact': 1684, '/home': 1649, '/checkout': 1639, '/about': 1741, '/products': 1679})
Average Session Duration per User: {'user_090': 169.90526315789472, 'user_029': 160.67768595041323, 'user_078': 161.17045454545453, 'user_028': 148.47619047619048, 'user_040': 143.05102040816325, 'user_096': 168.8190476190476, 'user_063': 157.63106796116506, 'user_051': 159.9090909090909, 'user_024': 155.89565217391305, 'user_066': 157.62222222222223, 'user_019': 158.0782608695652, 'user_080': 148.8252427184466, 'user_056': 164.51041666666666, 'user_037': 154.16190476190476, 'user_094': 149.46153846153845, 'user_086': 157.65853658536585, 'user_021': 142.56, 'user_010': 136.55652173913